## Test Framework Demo
1. Define map (`CENTRE`, `RES`), number of targets (`NUM_CASUALTY`), and number of search iterations (`STEPS`)
2. Instantiate TestFramework object
3. Register pathfinder
4. Generate casualty
5. Run test
6. Evaluate
7. Visualize

In [1]:
! pip install ..

Processing /Users/ivanfengjunkai/Desktop/SUTD/Term7/capstone/path-algo
  Preparing metadata (setup.py) ... done
  Created wheel for pathfinder: filename=pathfinder-0.1-py3-none-any.whl size=11134 sha256=66080c118732a6e75ccd7161a590664260b9467e1ee8abe420e8c2aa977a2573
  Stored in directory: /Users/ivanfengjunkai/Library/Caches/pip/wheels/79/f3/92/280141d26f35c80a2492bc052fe6c91cc2861379849ac0b08d
Successfully built pathfinder
  Attempting uninstall: pathfinder
    Found existing installation: pathfinder 0.1
    Uninstalling pathfinder-0.1:
      Successfully uninstalled pathfinder-0.1


In [2]:
import h3
import folium
import numpy as np
import os
import random

from pathfinder.algo import (
    OutwardSpiralPathFinder, 
    BayesianHexSearch
)
from utils.hex import (
    hex_to_binary, 
    octal_list_to_binary, 
    binary_to_octal_list, 
    binary_to_hex, 
    hex_to_array_index,
    array_index_to_hex,
    distance_between_2_hexas,
)
from utils.viz import (
    gradient_color,
    add_hex_to_map,
    visualise_hex_dict_to_map,
    create_gif
)
from experiment.test_framework import TestFramework

CENTRE = (1.340640225800291, 103.962424829341)
HOTSPOT1 = (1.340647264373615, 103.9623395934375)
HOTSPOT2 = (1.340590906845766, 103.96242415287534)
RES = 15
NUM_CASUALTY = 5
STEPS = 200
random.seed(0)

## Spiral

In [3]:
test_spiral = TestFramework(name="Spiral", centre=CENTRE, res=RES)
test_spiral.register_pathfinder(OutwardSpiralPathFinder)

test_spiral.add_hotspot(HOTSPOT1)
test_spiral.add_hotspot(HOTSPOT2)

test_spiral.generate_casualty(NUM_CASUALTY)
spiral_output = test_spiral.run(STEPS)
test_spiral.evaluate()



Spiral's Path Coverage: 27.74%
Spiral's Angle Curvature(Average angle): 13.331117027917337 degrees
Spiral's Guaranteed Capture: NO, captured 1/5
Spiral's Minimum Time Capture: NA


In [4]:
# visualise_hex_dict_to_map(test_spiral.probability_map, m, test_spiral.casualty_locations)
m = folium.Map(location=[CENTRE[0],CENTRE[1]], zoom_start=23, max_zoom=25)
hex_hotspot1 = h3.geo_to_h3(HOTSPOT1[0],HOTSPOT1[1], resolution = RES)
hex_hotspot2 = h3.geo_to_h3(HOTSPOT2[0],HOTSPOT2[1], resolution = RES)

visualise_hex_dict_to_map(test_spiral.probability_map, m, {hex_hotspot1:1, hex_hotspot2:1})

display(m)

In [5]:
add_hex_to_map(spiral_output, m)
display(m)

## Bayes

In [6]:
f = 0.99
test_bayes = TestFramework(name="Bayes", centre=CENTRE, res=RES, f=f)
test_bayes.register_pathfinder(BayesianHexSearch)

test_bayes.add_hotspot(HOTSPOT1)
test_bayes.add_hotspot(HOTSPOT2)

test_bayes.generate_casualty(NUM_CASUALTY)
bayes_output = test_bayes.run(STEPS, update_map=True)
test_bayes.evaluate()

m = folium.Map(location=[CENTRE[0],CENTRE[1]], zoom_start=23, max_zoom=25)

visualise_hex_dict_to_map(test_bayes.probability_map, m, test_bayes.casualty_locations)
add_hex_to_map(bayes_output, m)

display(m)

Bayes's Path Coverage: 27.74%
Bayes's Angle Curvature(Average angle): 30.94958973699486 degrees
Bayes's Guaranteed Capture: NO, captured 4/5
Bayes's Minimum Time Capture: NA


## Create GIF

In [7]:
if STEPS >= 200:
    print("This gna take forever")
output_path = os.path.join(os.getcwd(), "Spiral_path.gif")
create_gif(output_path, test_spiral.probability_map, spiral_output, test_spiral.casualty_locations, test_spiral.casualty_detected, dpi=50)
print("Spiral_path.gif done")
output_path = os.path.join(os.getcwd(), "Bayes_path.gif")
create_gif(output_path, test_bayes.probability_map, bayes_output, test_bayes.casualty_locations, test_bayes.casualty_detected, dpi=50)
print("Bayes_path.gif done")

This gna take forever


Ploting 721th image on map: 100%|██████████| 721/721 [00:15<00:00, 45.14it/s] 


Generate base map done!


Overlay hex 199 on map: : 200it [00:04, 42.32it/s]


Spiral_path.gif done


Ploting 721th image on map: 100%|██████████| 721/721 [00:15<00:00, 45.93it/s] 


Generate base map done!


Overlay hex 199 on map: : 200it [00:04, 43.45it/s]


Bayes_path.gif done
